In [4]:
import os
import cv2
import numpy as np
from tqdm import tqdm

training_data = np.load("training_data.npy", allow_pickle = True)

In [5]:
print(len(training_data))

24946


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        
        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)   
        
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512,2)
        
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)),(2,2))
        
        if self._to_linear is None:
            print(x[0].shape)
            self._to_linear = x[0].shape[0] * x[0].shape[1] * x[0].shape[2]
        return x
    
    def forward(self,x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = (self.fc2(x))
        return F.softmax(x, dim = 1)


net = Net()

torch.Size([128, 2, 2])


In [18]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr = 0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in  training_data]).view(-1, 50, 50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(X) * VAL_PCT)
print(val_size)

2494


In [19]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

print(len(train_X))
print(len(test_y))

22452
2494


In [24]:
BATCH_SIZE = 100
EPOCHS = 1

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
        batch_X = train_X[i:i + BATCH_SIZE].view(-1,1,50,50)
        batch_y = train_y[i:i + BATCH_SIZE]
        
        net.zero_grad()
        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
print(loss)


100%|██████████| 225/225 [00:50<00:00,  4.45it/s]


tensor(0.2038, grad_fn=<MseLossBackward>)


In [25]:
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1, 1, 50, 50))[0] 
        
        predicted_class = torch.argmax(net_out)
        if predicted_class == real_class:
            correct += 1
        total+=1

print("Accuracy:", round(correct/total, 3))
    


100%|██████████| 2494/2494 [00:05<00:00, 492.89it/s]

Accuracy: 0.613


In [28]:
net(test_X[i].view(-1, 1, 50, 50))

tensor([[0.3738, 0.6262]], grad_fn=<SoftmaxBackward>)

In [29]:
torch.cuda.is_available()

True

In [30]:
device = torch.device("cuda:0")

In [31]:
device

device(type='cuda', index=0)

In [33]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("running on cuda")
else: 
    device = torhc.device("cpu")
    print("running on cpu")

running on cuda


In [34]:
torch.cuda.device_count()

1

In [35]:
net.to(device)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

In [47]:
EPOCHS = 100
BATCH_SIZE = 100

def train(net):
    optimizer = optim.Adam(net.parameters(), lr = 0.001)
    loss_function = nn.MSELoss()
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
            batch_X = train_X[i:i + BATCH_SIZE].view(-1,1,50,50)
            batch_y = train_y[i:i + BATCH_SIZE]
            batch_X  = batch_X.to(device)
            batch_y  = batch_y.to(device)
            net.zero_grad()
            outputs = net(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()
#             print(f"Epoch: {epoch}. Loss: {loss}")
            
train(net)        
        


100%|██████████| 225/225 [00:02<00:00, 82.96it/s]

100%|██████████| 225/225 [00:02<00:00, 86.55it/s]

100%|██████████| 225/225 [00:02<00:00, 84.64it/s]

100%|██████████| 225/225 [00:02<00:00, 86.51it/s]

100%|██████████| 225/225 [00:02<00:00, 85.38it/s]

100%|██████████| 225/225 [00:02<00:00, 87.31it/s]

100%|██████████| 225/225 [00:02<00:00, 86.34it/s]

100%|██████████| 225/225 [00:02<00:00, 86.35it/s]

100%|██████████| 225/225 [00:02<00:00, 86.91it/s]

100%|██████████| 225/225 [00:02<00:00, 86.16it/s]

100%|██████████| 225/225 [00:02<00:00, 85.67it/s]

100%|██████████| 225/225 [00:02<00:00, 86.29it/s]

 28%|██▊       | 63/225 [00:00<00:01, 86.41it/s]


100%|██████████| 225/225 [00:02<00:00, 86.90it/s]

100%|██████████| 225/225 [00:02<00:00, 85.42it/s]

100%|██████████| 225/225 [00:02<00:00, 87.25it/s]

100%|██████████| 225/225 [00:02<00:00, 86.14it/s]

100%|██████████| 225/225 [00:02<00:00, 86.10it/s]

100%|██████████| 225/225 [00:02<00:00, 84.40it/s]

100%|██████████| 225/225 [00:02<00:00, 85.84it/s]

100%|██████████| 225/225 [00:02<00:00, 85.27it/s]

100%|██████████| 225/225 [00:02<00:00, 84.07it/s]

100%|██████████| 225/225 [00:02<00:00, 86.40it/s]

100%|██████████| 225/225 [00:02<00:00, 85.91it/s]

100%|██████████| 225/225 [00:02<00:00, 86.65it/s]

 48%|████▊     | 109/225 [00:01<00:01, 83.32it/s]


100%|██████████| 225/225 [00:02<00:00, 84.31it/s]

100%|██████████| 225/225 [00:02<00:00, 84.16it/s]

100%|██████████| 225/225 [00:02<00:00, 84.65it/s]

100%|██████████| 225/225 [00:02<00:00, 85.57it/s]

100%|██████████| 225/225 [00:02<00:00, 85.46it/s]

100%|██████████| 225/225 [00:02<00:00, 85.96it/s]

100%|██████████| 225/225 [00:02<00:00, 84.90it/s]

100%|██████████| 225/225 [00:02<00:00, 84.81it/s]

100%|██████████| 225/225 [00:02<00:00, 85.39it/s]

100%|██████████| 225/225 [00:02<00:00, 85.20it/s]

100%|██████████| 225/225 [00:02<00:00, 85.36it/s]

100%|██████████| 225/225 [00:02<00:00, 85.21it/s]

 68%|██████▊   | 153/225 [00:01<00:00, 84.28it/s]


100%|██████████| 225/225 [00:02<00:00, 84.75it/s]

100%|██████████| 225/225 [00:02<00:00, 84.87it/s]

100%|██████████| 225/225 [00:02<00:00, 85.47it/s]

100%|██████████| 225/225 [00:02<00:00, 85.46it/s]

100%|██████████| 225/225 [00:02<00:00, 85.29it/s]

100%|██████████| 225/225 [00:02<00:00, 86.29it/s]

100%|██████████| 225/225 [00:02<00:00, 85.70it/s]

100%|██████████| 225/225 [00:02<00:00, 85.54it/s]

100%|██████████| 225/225 [00:02<00:00, 85.97it/s]

100%|██████████| 225/225 [00:02<00:00, 84.29it/s]

100%|██████████| 225/225 [00:02<00:00, 84.91it/s]

100%|██████████| 225/225 [00:02<00:00, 84.26it/s]

 88%|████████▊ | 198/225 [00:02<00:00, 86.29it/s]


100%|██████████| 225/225 [00:02<00:00, 86.23it/s]

100%|██████████| 225/225 [00:02<00:00, 85.23it/s]

100%|██████████| 225/225 [00:02<00:00, 84.01it/s]

100%|██████████| 225/225 [00:02<00:00, 86.20it/s]

100%|██████████| 225/225 [00:02<00:00, 85.91it/s]

100%|██████████| 225/225 [00:02<00:00, 85.10it/s]

100%|██████████| 225/225 [00:02<00:00, 84.85it/s]

100%|██████████| 225/225 [00:02<00:00, 84.93it/s]

100%|██████████| 225/225 [00:02<00:00, 84.22it/s]

100%|██████████| 225/225 [00:02<00:00, 84.41it/s]

100%|██████████| 225/225 [00:02<00:00, 84.94it/s]

100%|██████████| 225/225 [00:02<00:00, 84.18it/s]

 92%|█████████▏| 207/225 [00:02<00:00, 85.51it/s]


100%|██████████| 225/225 [00:02<00:00, 85.84it/s]

100%|██████████| 225/225 [00:02<00:00, 84.11it/s]

100%|██████████| 225/225 [00:02<00:00, 84.77it/s]

100%|██████████| 225/225 [00:02<00:00, 84.77it/s]

100%|██████████| 225/225 [00:02<00:00, 85.71it/s]

100%|██████████| 225/225 [00:02<00:00, 84.30it/s]

100%|██████████| 225/225 [00:02<00:00, 83.84it/s]

100%|██████████| 225/225 [00:02<00:00, 84.15it/s]

100%|██████████| 225/225 [00:02<00:00, 85.14it/s]

100%|██████████| 225/225 [00:02<00:00, 83.84it/s]

100%|██████████| 225/225 [00:02<00:00, 84.10it/s]

100%|██████████| 225/225 [00:02<00:00, 85.20it/s]

100%|██████████| 225/225 [00:02<00:00, 83.80it/s]

  8%|▊         | 18/225 [00:00<00:02, 86.74it/s]


100%|██████████| 225/225 [00:02<00:00, 84.31it/s]

100%|██████████| 225/225 [00:02<00:00, 85.58it/s]

100%|██████████| 225/225 [00:02<00:00, 84.45it/s]

100%|██████████| 225/225 [00:02<00:00, 85.63it/s]

100%|██████████| 225/225 [00:02<00:00, 84.82it/s]

100%|██████████| 225/225 [00:02<00:00, 84.71it/s]

100%|██████████| 225/225 [00:02<00:00, 84.02it/s]

100%|██████████| 225/225 [00:02<00:00, 84.21it/s]

100%|██████████| 225/225 [00:02<00:00, 84.98it/s]

100%|██████████| 225/225 [00:02<00:00, 84.34it/s]

100%|██████████| 225/225 [00:02<00:00, 83.64it/s]

100%|██████████| 225/225 [00:02<00:00, 84.70it/s]

 16%|█▌        | 36/225 [00:00<00:02, 85.94it/s]


100%|██████████| 225/225 [00:02<00:00, 83.91it/s]

100%|██████████| 225/225 [00:02<00:00, 85.34it/s]

100%|██████████| 225/225 [00:02<00:00, 84.03it/s]

100%|██████████| 225/225 [00:02<00:00, 85.33it/s]

100%|██████████| 225/225 [00:02<00:00, 85.36it/s]

100%|██████████| 225/225 [00:02<00:00, 83.92it/s]

100%|██████████| 225/225 [00:02<00:00, 84.23it/s]

100%|██████████| 225/225 [00:02<00:00, 85.43it/s]

100%|██████████| 225/225 [00:02<00:00, 83.93it/s]

100%|██████████| 225/225 [00:02<00:00, 84.14it/s]

100%|██████████| 225/225 [00:02<00:00, 85.45it/s]

100%|██████████| 225/225 [00:02<00:00, 85.20it/s]

 28%|██▊       | 63/225 [00:00<00:01, 86.32it/s]


100%|██████████| 225/225 [00:02<00:00, 84.66it/s]

100%|██████████| 225/225 [00:02<00:00, 84.04it/s]

100%|██████████| 225/225 [00:02<00:00, 84.37it/s]


In [48]:
def test(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(test_X))):
            real_class = torch.argmax(test_y[i]).to(device)
            net_out = net(test_X[i].view(-1, 1, 50, 50).to(device))[0]

            predicted_class = torch.argmax(net_out)
            if predicted_class == real_class:
                correct += 1
            total+=1

    print("Accuracy:", round(correct/total, 3))

test(net)


100%|██████████| 2494/2494 [00:03<00:00, 626.99it/s]

Accuracy: 0.714


In [ ]:
def fwd_pass(X, y, train= False): #train Flag
    if train = True: 
        net.zero_grad()
    outputs = net(X)
    matches = [torch.argmax(i) == torch.argmax(j) for i, j in zip(outputs, y)]
    acc = matches.count(True)/len(matches)
    loss = loss_function(outputs, y)
    
    if train:
        loss.backward()
        optimizer.step()
    return acc, loss
    
        

In [ ]:
def test(size = 32):
    X, y = test_X[:size], test_y[:size]
    